In [1]:
from pyspark.sql.types import LongType, FloatType, StringType
from pyspark.sql.functions import col, udf, desc
import re
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('career').getOrCreate()

In [3]:
json_dataset = "marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson"
df = spark.read.json(json_dataset)

In [4]:
df.limit(10).toPandas().head()

,city,company_description,company_name,contact_email,country,crawl_timestamp,domain,duplicate_status,fitness_score,geo,...,post_date,postdate_in_indexname_format,postdate_yyyymmdd,salary_offered,site_name,state,test_contact_email,uniq_id,url,valid_through
0,Houston,,GPAC,None,US,2020-06-26 01:54:03 +0000,careerbuilder_com,NA,10,usa,...,2020-06-25,2020.06.22,20200625,,careerbuilder_usa_com_job_board_common_setup_p...,TX,None,9a457ef257fecf231693a6ba08f50293,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,2020-07-24
1,Cincinnati,None,Amazon Fulfillment,None,US,2020-05-17 01:21:05 +0000,careerbuilder_com,NA,10,usa,...,2020-05-16,2020.05.11,20200516,,careerbuilder_usa_com_job_board_common_setup_p...,OH,None,ba471e2faf6f79caf22cddebbedbc0e8,https://www.careerbuilder.com/job/J3P0FL6X795R...,2020-06-15
2,Peabody,None,Amazon Fulfillment,None,US,2020-06-27 04:53:42 +0000,careerbuilder_com,NA,10,usa,...,2020-06-26,2020.06.22,20200626,,careerbuilder_usa_com_job_board_common_setup_p...,MA,None,6f00bd02d63c633b5af453366f25c21e,https://www.careerbuilder.com/job/J3W3XK69NHHR...,2020-07-25
3,Villard,,Manpower,None,US,2020-06-03 01:21:32 +0000,careerbuilder_com,NA,10,usa,...,2020-06-02,2020.06.01,20200602,$14.00/ hour,careerbuilder_usa_com_job_board_common_setup_p...,MN,None,8ad0d00bfa23cfd7b7c364b8ae72085f,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,2020-07-01
4,Anchorage,,The North West Company - U.S.,None,US,2020-05-23 01:19:07 +0000,careerbuilder_com,NA,10,usa,...,2020-05-22,2020.05.22,20200522,,careerbuilder_usa_com_job_board_common_setup_p...,AK,None,31753dc342a1b2a07db712454c0d5f87,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,2020-06-21


### calculate number of jobs posted on daily basis, per each city

In [5]:
jobs_per_date_nd_each_state_df = df.groupBy(["post_date","city"]).count()
jobs_per_date_nd_each_state_df.write.option('header',True).csv('jobs_per_date_nd_each_state.csv')
jobs_per_date_nd_each_state_df.limit(10).toPandas().head()

,post_date,city,count
0,2020-04-28,East Brunswick,2
1,2020-06-12,Baton Rouge,2
2,2020-06-23,Belmont,4
3,2020-06-01,Goodyear,1
4,2020-06-17,King of Prussia,1


### calculate average salary per job title and state

In [6]:
HOURS_IN_YEAR = 8760

In [7]:
def return_yearly_salary(salary: str)-> float:
    """
    Extract annual salary
    args: salary - salary in different formats.
    return: annual salary in float.
    """
    avg_yearly_salary = 0.0
    salary = salary.replace(",","")
    if "$" in salary:
        if "." in salary:
            # float values filter
            digit_filter = "\d+\.\d+"
        else:
            # int values filter
            digit_filter = "\d+"
            
        all_digits = re.findall(digit_filter, salary)
        if re.search('hour',salary,re.IGNORECASE):
            avg_hourly_salary = sum(float(digit) for digit in all_digits)/len(all_digits)
            avg_yearly_salary = float(avg_hourly_salary * HOURS_IN_YEAR)
        else:
            avg_yearly_salary = sum(float(digit) for digit in all_digits) / len(all_digits)
            
    return avg_yearly_salary

In [8]:
yearly_udf = udf(lambda z: return_yearly_salary(z), FloatType())

In [9]:
df = df.withColumn("cleaned_yearly_salary", yearly_udf(col("salary_offered")))

In [10]:
average_salary_per_job_title_and_state_df = df.filter(df.cleaned_yearly_salary!=0.0).groupBy(["job_title","state"]).avg("cleaned_yearly_salary")
average_salary_per_job_title_and_state_df.write.option('header',True).csv('average_salary_per_job_title_and_state.csv')
average_salary_per_job_title_and_state_df.limit(10).toPandas().head()

,job_title,state,avg(cleaned_yearly_salary)
0,Sr. Accountant,TX,284700.0
1,"Private Equity Accountant - Boston, MA",MA,85000.0
2,Inbound Sales Opportunity - Work at Home,FL,42050.0
3,Retail Merchandiser,MN,111690.0
4,Retail Merchandiser,PA,111690.0


### Identify the top 10 most active companies by number of positions opened

In [11]:
top_10_most_active_companies_by_number_of_positions_opened_df = df.filter(df.has_expired=='false').groupBy("company_name").count().sort(desc("count")).limit(10)
top_10_most_active_companies_by_number_of_positions_opened_df.write.option('header',True).csv('top_10_most_active_companies_by_number_of_positions_opened.csv')

In [12]:
top_10_most_active_companies_by_number_of_positions_opened_df.limit(10).toPandas()

,company_name,count
0,Amazon Fulfillment,7501
1,GPAC,924
2,LanguageLine Solutions,871
3,Lowe's Home Improvement,861
4,CDL Career Now,545
5,Advance Auto Parts,492
6,Aerotek,477
7,Robert Half,444
8,Driveline Retail,372
9,Circle K,334


### Create a UDF function to clean job description from HTML code contained inside

In [13]:
# source: https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html: str) -> str:
    """
    Code to clean the html
    args: raw_html - html in string format.
    return: cleaned html text
    """
    if type(raw_html)!=str:
        return ""
    cleantext = re.sub(CLEANR, '', raw_html)
    cleantext = cleantext.replace('/>','.')
    return cleantext

In [14]:
html_cleaner_udf = udf(lambda z: cleanhtml(z),StringType())

df = df.filter(df.job_description.isNotNull())

cleaned_job_description_df = df.withColumn("cleaned_job_description", html_cleaner_udf(col("job_description")))
cleaned_job_description_df.write.option('header',True).csv('html_cleaned_job_description.csv')


In [15]:
cleaned_job_description_df.limit(10).toPandas().head()

,city,company_description,company_name,contact_email,country,crawl_timestamp,domain,duplicate_status,fitness_score,geo,...,postdate_yyyymmdd,salary_offered,site_name,state,test_contact_email,uniq_id,url,valid_through,cleaned_yearly_salary,cleaned_job_description
0,Houston,,GPAC,None,US,2020-06-26 01:54:03 +0000,careerbuilder_com,NA,10,usa,...,20200625,,careerbuilder_usa_com_job_board_common_setup_p...,TX,None,9a457ef257fecf231693a6ba08f50293,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,2020-07-24,0.0,SR. PROJECT MANAGER WANTED!!! My client is a M...
1,Cincinnati,None,Amazon Fulfillment,None,US,2020-05-17 01:21:05 +0000,careerbuilder_com,NA,10,usa,...,20200516,,careerbuilder_usa_com_job_board_common_setup_p...,OH,None,ba471e2faf6f79caf22cddebbedbc0e8,https://www.careerbuilder.com/job/J3P0FL6X795R...,2020-06-15,0.0,"Shifts: Overnight, Early Morning, Day, Evening..."
2,Peabody,None,Amazon Fulfillment,None,US,2020-06-27 04:53:42 +0000,careerbuilder_com,NA,10,usa,...,20200626,,careerbuilder_usa_com_job_board_common_setup_p...,MA,None,6f00bd02d63c633b5af453366f25c21e,https://www.careerbuilder.com/job/J3W3XK69NHHR...,2020-07-25,0.0,"Shifts: Early Morning, Day, Evening Location: ..."
3,Villard,,Manpower,None,US,2020-06-03 01:21:32 +0000,careerbuilder_com,NA,10,usa,...,20200602,$14.00/ hour,careerbuilder_usa_com_job_board_common_setup_p...,MN,None,8ad0d00bfa23cfd7b7c364b8ae72085f,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,2020-07-01,122640.0,Do you pride yourself on attention to detail a...
4,Anchorage,,The North West Company - U.S.,None,US,2020-05-23 01:19:07 +0000,careerbuilder_com,NA,10,usa,...,20200522,,careerbuilder_usa_com_job_board_common_setup_p...,AK,None,31753dc342a1b2a07db712454c0d5f87,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,2020-06-21,0.0,In your role as Graphics Designer for Alaska C...
